In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-e6cadkn0
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-e6cadkn0
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=7563e7e036e4fd9f315eaf0255e70b104123a33fbe3527cdc5d494ddce4ac9d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-swzsrego/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [7]:
%%cuda --name my_curand.cu 

#include <iostream>
#include <cuda_runtime.h>
#include <cuda_runtime_api.h>

#include <iostream>
#include <cmath>
#include <vector>

#include </content/src/ParallelForGPU.h>

using namespace std;

HOST DEVICE
inline void test_function(int i, int j, int k, 
                          Array4<double> const &vel,
													Array4<double> const &pressure) {
	vel(i, j, k) = i+j+k;
	pressure(i,j,k) = 2*i*j;
}

int main(){
	
	int nx = 5, ny = 4, nz = 3;
  
	MultiFab velfab(nx, ny, nz);
	MultiFab pressurefab(nx, ny, nz);

	auto vel = velfab.array();
	auto pressure = pressurefab.array();
  
	ParallelFor(nx, ny, nz,
	[=] DEVICE (int i, int j, int k) noexcept
	{
		test_function(i, j, k, vel, pressure);
	});

cudaDeviceProp prop;
cudaGetDeviceProperties(&prop, 0);
printf("Device name: %s\n", prop.name);
printf("Total global memory: %lu bytes\n", prop.totalGlobalMem);
printf("Shared memory per block: %lu bytes\n", prop.sharedMemPerBlock);
printf("Maximum threads per block: %d\n", prop.maxThreadsPerBlock);
printf("Clock rate: %d kHz\n", prop.clockRate);

int device;
    cudaGetDevice(&device);

    int mp_count;
    cudaDeviceGetAttribute(&mp_count, cudaDevAttrMultiProcessorCount, device);

    int max_threads_per_mp;
    cudaDeviceGetAttribute(&max_threads_per_mp, cudaDevAttrMaxThreadsPerMultiProcessor, device);

    int total_threads = mp_count * max_threads_per_mp;
    printf("Total number of threads on device %d: %d %d %d\n", device, mp_count, max_threads_per_mp, total_threads);



	cudaDeviceSynchronize();

	for(int i=0;i<nx;i++){
		for(int j=0;j<ny;j++){
			for(int k=0;k<nz;k++){
				cout << "Vel at " << i << "," << j << "," << k << " is " << vel(i,j,k) << " " << pressure(i,j,k) << "\n";
			}
		}
	}

return 0;
}


'File written in /content/src/my_curand.cu'

In [3]:
%%cuda --name ParallelForGPU.h

#define HOST __host__
#define DEVICE __device__

#define GPU_MAX_THREADS 512

#define LAUNCH_KERNEL(MT, blocks, threads, sharedMem, ... ) \
        launch_global<MT><<<blocks, threads, sharedMem>>>(__VA_ARGS__)

template<typename T>
struct Array4{
	T* data;
	int jstride;
	int kstride;

  constexpr Array4(T* a_p): data(a_p){};

	public:
    	__host__ __device__
		T& operator()(int i, int j, int k)const noexcept{
						return data[i + j*jstride + k*kstride];
		}
};

template<int launch_bounds_max_threads, class L>
__launch_bounds__(launch_bounds_max_threads)
__global__ void launch_global (L f0) { f0(); }

template <typename F>
DEVICE
auto call_f(F const &f, int i, int j, int k){
	f(i, j, k);
}

template<class L>
void ParallelFor(int nx, int ny, int nz, L &&f){
		int len_xy = nx*ny;
		int len_x = nx;
		int ncells = nx*ny*nz;
		int numBlocks = (std::max(ncells,1) + GPU_MAX_THREADS - 1 )/GPU_MAX_THREADS;
		int numThreads = GPU_MAX_THREADS;
		std::cout << "Launching " << numBlocks << " blocks " << "\n";
		LAUNCH_KERNEL(GPU_MAX_THREADS, numBlocks, numThreads, 0,
    		[=] DEVICE () noexcept{	
			for(int icell = blockDim.x*blockIdx.x+threadIdx.x, stride = blockDim.x*gridDim.x;
        	icell < nx*ny*nz; icell += stride){
				int k = icell/len_xy;
				int j = (icell - k*len_xy)/len_x;
				int i = (icell - k*len_xy - j*len_x); 
				call_f(f, i, j, k);	
			}
		});
}

class MultiFab{
		
		int nx, ny, nz;

		public:
		
		MultiFab(int a_nx, int a_ny, int a_nz): nx(a_nx), ny(a_ny), nz(a_nz){};

		Array4<double> array()
		{
				Array4<double> *vec;
  			cudaMallocManaged((void**)&vec, sizeof(Array4<double>));
  			cudaMallocManaged((void**)&(vec[0].data), nx*ny*nz*sizeof(double));
				vec[0].jstride = nx;
				vec[0].kstride = nx*ny;
				return vec[0];
		}		
};

'File written in /content/src/ParallelForGPU.h'

In [4]:
%%cuda --name ParallelForCPU.h

#define HOST 
#define DEVICE 

template<typename T>
struct Array4{
	T* data;
	int jstride;
	int kstride;

  constexpr Array4(T* a_p): data(a_p){};

	public:
    	__host__ __device__
		T& operator()(int i, int j, int k)const noexcept{
						return data[i + j*jstride + k*kstride];
		}
};

template <typename F>
auto call_f(F const &f, int i, int j, int k){
	f(i, j, k);
}

template<class L>
void ParallelFor(int nx, int ny, int nz, L &&f){
	for(int i=0;i<nx;i++){
	  for(int j=0;j<ny;j++){
				for(int k=0;k<nz;k++){
				call_f(f, i, j, k);	
			}
    }
  }
}

class MultiFab{
		
		int nx, ny, nz;

		public:
		
		MultiFab(int a_nx, int a_ny, int a_nz): nx(a_nx), ny(a_ny), nz(a_nz){};

		Array4<double> array()
		{
				Array4<double> *vec;
        vec = (Array4<double>*)malloc(sizeof(Array4<double>));
        vec[0].data = (double*)malloc(nx*ny*nz*sizeof(double));
				vec[0].jstride = nx;
				vec[0].kstride = nx*ny;
				return vec[0];
		}		
};

'File written in /content/src/ParallelForCPU.h'

In [9]:
!nvcc -expt-extended-lambda --expt-relaxed-constexpr --forward-unknown-to-host-compiler --Werror ext-lambda-captures-this -Xcudafe --diag_suppress=esa_on_defaulted_function_ignored -o /content/src/my_curand /content/src/my_curand.cu -lcurand


In [10]:
!compute-sanitizer /content/src/my_curand


========= COMPUTE-SANITIZER
Launching 1 blocks 
Device name: Tesla T4
Total global memory: 15835398144 bytes
Shared memory per block: 49152 bytes
Maximum threads per block: 1024
Clock rate: 1590000 kHz
Total number of threads on device 0: 40 1024 40960
Vel at 0,0,0 is 0 0
Vel at 0,0,1 is 1 0
Vel at 0,0,2 is 2 0
Vel at 0,1,0 is 1 0
Vel at 0,1,1 is 2 0
Vel at 0,1,2 is 3 0
Vel at 0,2,0 is 2 0
Vel at 0,2,1 is 3 0
Vel at 0,2,2 is 4 0
Vel at 0,3,0 is 3 0
Vel at 0,3,1 is 4 0
Vel at 0,3,2 is 5 0
Vel at 1,0,0 is 1 0
Vel at 1,0,1 is 2 0
Vel at 1,0,2 is 3 0
Vel at 1,1,0 is 2 2
Vel at 1,1,1 is 3 2
Vel at 1,1,2 is 4 2
Vel at 1,2,0 is 3 4
Vel at 1,2,1 is 4 4
Vel at 1,2,2 is 5 4
Vel at 1,3,0 is 4 6
Vel at 1,3,1 is 5 6
Vel at 1,3,2 is 6 6
Vel at 2,0,0 is 2 0
Vel at 2,0,1 is 3 0
Vel at 2,0,2 is 4 0
Vel at 2,1,0 is 3 4
Vel at 2,1,1 is 4 4
Vel at 2,1,2 is 5 4
Vel at 2,2,0 is 4 8
Vel at 2,2,1 is 5 8
Vel at 2,2,2 is 6 8
Vel at 2,3,0 is 5 12
Vel at 2,3,1 is 6 12
Vel at 2,3,2 is 7 12
Vel at 3,0,0 is 3 0
Vel 